Let's calculate binary and multi class log loss / cross-entropy for a sample dataset; note that this is not a neat implementation of either - I'm just working some things out to understand it better :) 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics

%matplotlib inline

Step 1: Whip up a sample dataset!

In [2]:
#these will be out predictions
y_pred = pd.DataFrame(np.random.uniform(0,1,10), columns = ["y_pred"])

In [3]:
#these will be the true values of the thing we're predicting
y = pd.DataFrame(np.random.randint(0,2,y_pred.size), columns = ["y"])

In [4]:
df = y_pred.join(y)
df

,y_pred,y
0,0.655722,0
1,0.099145,0
2,0.528746,0
3,0.436951,0
4,0.732424,0
5,0.735332,1
6,0.736707,1
7,0.251428,1
8,0.179033,0
9,0.990145,0


Calculate binary log loss for each row

In [5]:
df['logloss'] = - (df.y * np.log(df.y_pred) + (1 - df.y) * (np.log(1 - df.y_pred)))

In [6]:
df

,y_pred,y,logloss
0,0.655722,0,1.066306
1,0.099145,0,0.104411
2,0.528746,0,0.752357
3,0.436951,0,0.574389
4,0.732424,0,1.318352
5,0.735332,1,0.307434
6,0.736707,1,0.305565
7,0.251428,1,1.380599
8,0.179033,0,0.197272
9,0.990145,0,4.619819


In [7]:
avg_log_loss = df.logloss.mean()
avg_log_loss

1.062650534036272

Let's compare it with a log loss calculated with sklearn

In [8]:
sklearn.metrics.log_loss(y,y_pred)

1.062650534036272

In [9]:
assert avg_log_loss == sklearn.metrics.log_loss(y,y_pred)

In [10]:
Multi-class cross-entropy

SyntaxError: invalid syntax (<ipython-input-10-cb3676c7758a>, line 1)

Let's make an example of a Multi class log loss. In this case, let's prepare a dataset with 4 columns: one for the value of a 3-class thing we want to predict (which takes values from 1 to 3), and 3 variables we'll pretend are our predicted probabilities for each of these 3 clases.

In [ ]:
df = pd.DataFrame(np.random.uniform(0,1,10), columns = ["prob_y1"])
df["prob_y2"] = pd.DataFrame(np.random.uniform(0,1,10), columns = ["prob_y2"])
df["prob_y3"] = pd.DataFrame(np.random.uniform(0,1,10), columns = ["prob_y3"])
df["y"] = pd.DataFrame(np.random.randint(1,4,10), columns = ["y"])

In [ ]:
df

Let's normalize the probabilities, so they add up to 1 for each row

In [ ]:
sum_of_probs = df[["prob_y1","prob_y2","prob_y3"]].sum(axis=1) 

df["prob_y1"] /= sum_of_probs
df["prob_y2"] /= sum_of_probs
df["prob_y3"] /= sum_of_probs
df

Let's add separate 'y' columns for each label to make it apparent how multi-class log loss relates to binary log loss

In [ ]:
df["y1"] = df["y"] == 1
df["y2"] = df["y"] == 2
df["y3"] = df["y"] == 3
df

now, it's as simple as taking the negative average of the three binary log losses (without the minus sign)

In [ ]:
df['logloss_y1'] = - (df.y1 * np.log(df.prob_y1))
df['logloss_y2'] = - (df.y2 * np.log(df.prob_y2))
df['logloss_y3'] = - (df.y3 * np.log(df.prob_y3))
df['logloss'] = df[["logloss_y1", "logloss_y2", "logloss_y3"]].sum(axis=1)
df

Now we need to sum up all individual losses and take an average of them.

In [ ]:
_sum_logloss = df.logloss.sum()

In [ ]:
avg_logloss = _sum_logloss/df.shape[0]
avg_logloss

In [ ]:
sklearn.metrics.log_loss(df[["y1", "y2", "y3"]], df[["prob_y1", "prob_y2", "prob_y3"]])

boom